In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import glob
import os
from shapely.geometry import Polygon
from shapely.geometry import Point

### 노인 인구 밀집 지역- 경남

In [2]:
# Shapefile 파일이 있는 디렉토리 경로
directory_path = "../../data/경남 노인 인구 분포/"

# Shapefile 파일 경로를 모두 가져오기
shapefile_paths = glob.glob(os.path.join(directory_path, "**/*.shp"))

gdfs = []

# 각 Shapefile을 읽어 리스트에 추가
for shapefile in shapefile_paths:
    gdf = gpd.read_file(shapefile, encoding="utf-8")
    gdfs.append(gdf)

# 모든 GeoDataFrame을 하나로 병합
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


merged_gdf['val'] = merged_gdf['val'].fillna(0)
merged_gdf = merged_gdf[merged_gdf["val"].apply(lambda x: x != 0.0)]
merged_gdf = merged_gdf.to_crs(epsg=5179)

In [3]:
merged_gdf['point'] = merged_gdf['geometry'].centroid.apply(lambda centroid: Point(centroid.x, centroid.y))

In [17]:
merged_gdf["경도"] = gpd.GeoDataFrame(
     merged_gdf[["point"]], 
     geometry="point"
     ).to_crs(epsg=4236).point.x

merged_gdf["위도"] = gpd.GeoDataFrame(
     merged_gdf[["point"]], 
     geometry="point"
     ).to_crs(epsg=4236).point.y

In [20]:
# merged_gdf.to_csv("../../data/노인인구분포중심점.csv", index=False)